In [27]:
# @title Nerarest Neighbour with Collaborative Filtering

In [28]:
import pandas as pd
import numpy as np

In [29]:
movies = pd.read_csv("/content/movies.csv")
ratings = pd.read_csv("/content/ratings.csv")

In [30]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [31]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [32]:
#merging the two df
df = pd.merge(ratings,movies,on="movieId")
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [33]:
df.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0
title,0
genres,0


In [34]:
combine_movie_rating = df.dropna(axis=0,subset=['title'])

In [35]:
combine_movie_rating

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [36]:
movie_rating_count = (combine_movie_rating.groupby("title")['rating'].count().reset_index().rename(columns = {'rating':"totalRatingCount"}))[['totalRatingCount','title']]

In [37]:
movie_rating_count

,totalRatingCount,title
0,1,'71 (2014)
1,1,'Hellboy': The Seeds of Creation (2004)
2,2,'Round Midnight (1986)
3,1,'Salem's Lot (2004)
4,2,'Til There Was You (1997)
...,...,...
9714,22,eXistenZ (1999)
9715,24,xXx (2002)
9716,5,xXx: State of the Union (2005)
9717,26,¡Three Amigos! (1986)


In [38]:
rating_with_total_rating_count = combine_movie_rating.merge(movie_rating_count,left_on = "title",right_on = "title",how = "left")

In [39]:
rating_with_total_rating_count

,userId,movieId,rating,timestamp,title,genres,totalRatingCount
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
...,...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller,1
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama,1
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller,1
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller,1


In [40]:
rating_with_total_rating_count.totalRatingCount.value_counts()

,count
totalRatingCount,
1,3441
2,2596
3,2400
4,2116
7,1946
...,...
128,128
124,124
113,113


In [41]:
popularity_thershold = 50
rating_popular_movie = rating_with_total_rating_count.query('totalRatingCount>=@popularity_thershold')
rating_popular_movie

,userId,movieId,rating,timestamp,title,genres,totalRatingCount
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
...,...,...,...,...,...,...,...
79246,603,1997,4.0,953925513,"Exorcist, The (1973)",Horror|Mystery,53
79247,606,1997,3.0,1178911117,"Exorcist, The (1973)",Horror|Mystery,53
79248,607,1997,5.0,963079420,"Exorcist, The (1973)",Horror|Mystery,53
79249,608,1997,4.5,1117502891,"Exorcist, The (1973)",Horror|Mystery,53


In [42]:
movies_features_df = rating_popular_movie.pivot_table(index="title",columns = "userId",values = "rating").fillna(0)
movies_features_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X2: X-Men United (2003),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
You've Got Mail (1998),0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
Young Frankenstein (1974),5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0


In [43]:
#converting the pivot into array
import scipy
from scipy.sparse import csr_matrix
movies_features_df_matrix = csr_matrix(movies_features_df.values)

In [44]:
movies_features_df_matrix

<450x606 sparse matrix of type '<class 'numpy.float64'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [45]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = "cosine",algorithm ="brute")
model_knn.fit(movies_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [46]:
query_index=np.random.choice(movies_features_df.shape[0])
print(query_index)

62


In [47]:
distances, indices = model_knn.kneighbors(movies_features_df.iloc[query_index,:].values.reshape(1,-1), n_neighbors=6)

In [48]:
distances

array([[0.        , 0.46032796, 0.4624649 , 0.47236837, 0.47587738,
        0.48548256]])

In [49]:
indices

array([[ 62, 123, 109,   4,  68,  85]])

In [50]:
for i in range(0,len(distances.flatten())):
  if i==0:
    print('Recommendation for {0}\n'.format(movies_features_df.index[query_index]))
  else:

    print('{0}: {1}, with distance of {2}:'.format(i, movies_features_df.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendation for Blood Diamond (2006)

1: Departed, The (2006), with distance of 0.4603279617800168:
2: Crash (2004), with distance of 0.4624649033648578:
3: 300 (2007), with distance of 0.4723683662432485:
4: Bourne Ultimatum, The (2007), with distance of 0.4758773844631554:
5: Catch Me If You Can (2002), with distance of 0.4854825621142532:
